In [1]:
% matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
H1_data = np.genfromtxt('H1_times.csv')
L1_data = np.genfromtxt('L1_times.csv')

ValueError: Some errors were detected !
    Line #672 (got 1 columns instead of 12)
    Line #673 (got 1 columns instead of 12)
    Line #674 (got 1 columns instead of 12)
    Line #675 (got 1 columns instead of 12)
    Line #676 (got 1 columns instead of 12)
    Line #677 (got 1 columns instead of 12)
    Line #678 (got 1 columns instead of 12)
    Line #679 (got 1 columns instead of 12)
    Line #680 (got 1 columns instead of 12)
    Line #681 (got 1 columns instead of 12)
    Line #682 (got 1 columns instead of 12)
    Line #683 (got 1 columns instead of 12)
    Line #1075 (got 1 columns instead of 12)
    Line #1076 (got 1 columns instead of 12)
    Line #1077 (got 1 columns instead of 12)
    Line #1078 (got 1 columns instead of 12)
    Line #1079 (got 1 columns instead of 12)
    Line #1080 (got 1 columns instead of 12)
    Line #1081 (got 1 columns instead of 12)
    Line #1082 (got 1 columns instead of 12)
    Line #1083 (got 1 columns instead of 12)
    Line #1084 (got 1 colum

In [ ]:
H1_cbc = H1_data[np.where(H1_data[:,9] == 1)]
H1_burst = H1_data[np.where(H1_data[:,10] == 1)]
H1_stoch = H1_data[np.where(H1_data[:,11] == 1)]

In [ ]:
# using multilabels, with i = 0 = cbc,
#                           = 1 = burst,
#                           = 2 = stoch,
#                           = 3 = terrestrial
training = np.array([])
target = np.array([])
for event in H1_data:
    if (event[9] == 1 or event[10] == 1 or event[11] == 1):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([int(event[9]),int(event[10]),int(event[11]),0])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target,np.array([int(event[9]),int(event[10]),int(event[11]),0])))
        
        continue # not counting hardware injections as terrestrial

    
    b = event[0]
    e = event[1]
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > L1_data[:,0]) & (b < L1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > L1_data[:,0]) & (e < L1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([0,0,0,1])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target, np.array([0,0,0,1])))
            
for event in L1_data:
    if (event[9] == 1 or event[10] == 1 or event[11] == 1):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([int(event[9]),int(event[10]),int(event[11]),0])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target,np.array([int(event[9]),int(event[10]),int(event[11]),0])))
        
        continue # not counting hardware injections as terrestrial

    
    b = event[0]
    e = event[1]
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > H1_data[:,0]) & (b < H1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > H1_data[:,0]) & (e < H1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([0,0,0,1])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target, np.array([0,0,0,1])))

In [ ]:
# scale data for mlp
scaler = StandardScaler()  

scaler.fit(training)
train_scale = scaler.transform(training)

In [ ]:
# split for training and testing
dat_train, dat_test, tar_train, tar_test = train_test_split(train_scale, target, test_size=0.4)

In [ ]:
# MLP params
hidden = (3,)
activ = 'logistic'
solver = 'lbfgs'
lrn = 'adaptive'

In [ ]:
mlp = MLPClassifier()
parameters = {'hidden_layer_sizes':[(3,)],'activation':['logistic'],'solver':['lbfgs'],learning_rate=['adaptive'],'alpha':(10.0 ** -np.arange(1, 7))}
clf = GridSearchCV(mlp, parameters)
clf.fit(dat_train,tar_train)

In [ ]:
alpha = clf.best_params_['alpha']

mlp = MLPClassifier(hidden_layer_sizes=hidden,activation=activ,solver=solver,learning_rate=lrn,alpha=alpha)
mlp.fit(dat_train,tar_train)

In [ ]:
mlp.score(dat_test,tar_test)

In [ ]:
scores = cross_val_score(mlp, train_scale, target, cv=10)

In [ ]:
print("Accuracy: %0.2f +/- %0.2f" % (scores.mean(), scores.std() * 2))

In [ ]:
print(np.sum(target, axis=0))

In [ ]:
H1 = np.array([H1_data[:,2],H1_data[:,3],H1_data[:,4],H1_data[:,5]]).T
L1 = np.array([L1_data[:,2],L1_data[:,3],L1_data[:,4],L1_data[:,5]]).T
print(H1.shape)
print()
print(train_scale.shape)